In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time

In [7]:
spark = SparkSession.builder.master('local[*]').appName('BookRate').getOrCreate()

In [8]:
rate_df = spark.read.csv('/content/Book-Ratings.csv', sep=';', header=True, inferSchema=True)

In [9]:
rate_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- rate: integer (nullable = true)



In [10]:
rate_df_filtered=rate_df.select('isbn', 'rate')
rate_df_filtered.show(5)

+----------+----+
|      isbn|rate|
+----------+----+
|034545104X|   0|
|0155061224|   5|
|0446520802|   0|
|052165615X|   3|
|0521795028|   6|
+----------+----+
only showing top 5 rows



In [12]:
result=rate_df_filtered.groupBy('isbn').agg(collect_list('rate').alias('rate')).orderBy('isbn')
result.show(10,truncate=False)

+-----------+------+
|isbn       |rate  |
+-----------+------+
| 0330299891|[0, 6]|
| 0375404120|[3, 0]|
| 0586045007|[0]   |
| 9022906116|[7, 0]|
| 9032803328|[0]   |
| 9044922564|[0]   |
| 9044922572|[0]   |
| 9044922718|[0]   |
| 9044923161|[0]   |
| 904492401X|[0]   |
+-----------+------+
only showing top 10 rows



In [15]:
str_time = time.time()
result.repartition(1).write.orc('/content/bookRate_output_orc')
dur = time.time() - str_time
print(dur)

10.935255289077759


In [16]:
result.printSchema()

root
 |-- isbn: string (nullable = true)
 |-- rate: array (nullable = false)
 |    |-- element: integer (containsNull = false)



In [17]:
result.show(5)

+-----------+------+
|       isbn|  rate|
+-----------+------+
| 0330299891|[0, 6]|
| 0375404120|[3, 0]|
| 0586045007|   [0]|
| 9022906116|[7, 0]|
| 9032803328|   [0]|
+-----------+------+
only showing top 5 rows



In [22]:
result2=result.select('isbn', explode('rate').alias('rate'))
result2.show(10,truncate=False)

+-----------+----+
|isbn       |rate|
+-----------+----+
| 0330299891|0   |
| 0330299891|6   |
| 0375404120|3   |
| 0375404120|0   |
| 0586045007|0   |
| 9022906116|7   |
| 9022906116|0   |
| 9032803328|0   |
| 9044922564|0   |
| 9044922572|0   |
+-----------+----+
only showing top 10 rows



In [24]:
str_time = time.time()
result2.repartition(1).write.orc('/content/bookRate_output2_orc')
dur = time.time() - str_time
print(dur)

12.328521490097046
